<a href="https://www.kaggle.com/code/jvthunder/sign-language-classification?scriptVersionId=121233415" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

This notebook is created for [Kaggle's Sign Langueage Classifier Competition].(https://www.kaggle.com/competitions/asl-signs/code)

In this notebook, I used notebooks from various sources and modify them to get this notebook. All credits are due to the authors of these notebooks and please give upvotes to them if you also find it useful.

- I used some of the data visualization of the landmark from [Sign Language EDA & Visualization] (https://www.kaggle.com/code/mayukh18/sign-language-eda-visualization).

- I used the preprocessed tensorflow Dataset from [tfdataset-of-google-isl-recognition-data] (https://www.kaggle.com/datasets/aapokossi/saved-tfdataset-of-google-isl-recognition-data)

- I train my model following the notebook. [Submission for variable length time-series model] (https://www.kaggle.com/code/aapokossi/submission-for-variable-length-time-series-model) I then tweak the layers and the epoch to increase the accuracy.

In future versions, I might improve on the model and give more comments to help people start on this competition. 

<a id="contents"></a>
# Contents
1. [Import Libraries and Set File Directories](#section-one)
2. [Visualize data](#section-two)
3. [Load Data](#section-three)
4. [Train Model](#section-four)
5. [Submit Model](#section-five)

<a id="section-one"></a>
# Import Libraries and Set File Directories

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, optimizers

In [ ]:
LANDMARK_FILES_DIR = "/kaggle/input/asl-signs/train_landmark_files"
TRAIN_FILE = "/kaggle/input/asl-signs/train.csv"

<a id="section-two"></a>
# Visualize data

In [ ]:
sample = pd.read_parquet("/kaggle/input/asl-signs/train_landmark_files/16069/100015657.parquet")
sample.head()

In [ ]:
sample_left_hand = sample[sample.type == "left_hand"]
sample_right_hand = sample[sample.type == "right_hand"]
edges = [(0,1),(1,2),(2,3),(3,4),(0,5),(0,17),(5,6),(6,7),(7,8),(5,9),(9,10),(10,11),(11,12),
         (9,13),(13,14),(14,15),(15,16),(13,17),(17,18),(18,19),(19,20)]

def plot_frame(df, frame_id, ax):
    df = df[df.frame == frame_id].sort_values(['landmark_index'])
    x = list(df.x)
    y = list(df.y)
    
    ax.scatter(df.x, df.y, color='dodgerblue')
    for i in range(len(x)):
        ax.text(x[i], y[i], str(i))
        
    for edge in edges:
        ax.plot([x[edge[0]], x[edge[1]]], [y[edge[0]], y[edge[1]]], color='salmon')
        ax.set_xlabel(f"Frame no. {frame_id}")
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])

    
def plot_frame_seq(df, frame_range, n_frames):
    frames = np.linspace(frame_range[0],frame_range[1],n_frames, dtype = int, endpoint=True)
    fig, ax = plt.subplots(n_frames, 1, figsize=(5,25))
    for i in range(n_frames):
        plot_frame(df, frames[i], ax[i])
        
    plt.show()

    
plot_frame_seq(sample_left_hand, (178,186), 5)

<a id="section-three"></a>
# Load Data

In [ ]:
# Set constants and pick important landmarks
LANDMARK_IDX = [0,9,11,13,14,17,117,118,119,199,346,347,348] + list(range(468,543))
DATA_PATH = "/kaggle/input/saved-tfdataset-of-google-isl-recognition-data/GoogleISLDatasetBatched"
DS_CARDINALITY = 185
VAL_SIZE  = 20
N_SIGNS = 250
ROWS_PER_FRAME = 543

In [ ]:
def preprocess(ragged_batch, labels):
    ragged_batch = tf.gather(ragged_batch, LANDMARK_IDX, axis=2)
    ragged_batch = tf.where(tf.math.is_nan(ragged_batch), tf.zeros_like(ragged_batch), ragged_batch)
    return tf.concat([ragged_batch[...,i] for i in range(3)],-1), labels

dataset = tf.data.Dataset.load(DATA_PATH)
dataset = dataset.map(preprocess)
val_ds = dataset.take(VAL_SIZE).cache().prefetch(tf.data.AUTOTUNE)
train_ds = dataset.skip(VAL_SIZE).cache().shuffle(20).prefetch(tf.data.AUTOTUNE)

<a id="section-four"></a>
# Train Model

In [ ]:
def get_callbacks():
    return [
            tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor = "val_accuracy",
            factor = 0.5,
            patience = 3
        ),
    ]

def dense_block(units, name):
    fc = layers.Dense(units)
    norm = layers.LayerNormalization()
    act = layers.Activation("relu")
    return lambda x: act(norm(fc(x)))

def classifier(lstm_units):
    lstm = layers.LSTM(lstm_units)
    out = layers.Dense(N_SIGNS, activation="softmax")
    return lambda x: out(lstm(x))

In [ ]:
encoder_units = [512, 256]
lstm_units = 256

#define the inputs (ragged batches of time series of landmark coordinates)
inputs = tf.keras.Input(shape=(None,3*len(LANDMARK_IDX)), ragged=True)

# dense encoder model
x = inputs
for i, n in enumerate(encoder_units):
    x = dense_block(n, f"encoder_{i}")(x)

# classifier model
out = classifier(lstm_units)(x)

model = tf.keras.Model(inputs=inputs, outputs=out)
model.summary()

In [ ]:
steps_per_epoch = DS_CARDINALITY - VAL_SIZE
boundaries = [steps_per_epoch * n for n in [30,50,70]]
values = [1e-3,1e-4,1e-5,1e-6]
lr_sched = optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
optimizer = optimizers.Adam(lr_sched)

model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy","sparse_top_k_categorical_accuracy"])

In [ ]:
model.fit(train_ds,
          validation_data = val_ds,
          callbacks = get_callbacks(),
          epochs = 100)

<a id="section-five"></a>
# Submit Model

In [ ]:
model.summary(expand_nested=True)

In [ ]:
def get_inference_model(model):
    inputs = tf.keras.Input(shape=(ROWS_PER_FRAME,3), name="inputs")
    
    # drop most of the face mesh
    x = tf.gather(inputs, LANDMARK_IDX, axis=1)

    # fill nan
    x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)

    # flatten landmark xyz coordinates ()
    x = tf.concat([x[...,i] for i in range(3)], -1)

    x = tf.expand_dims(x,0)
    
    # call trained model
    out = model(x)
    
    # explicitly name the final (identity) layer for the submission format
    out = layers.Activation("linear", name="outputs")(out)
    
    inference_model = tf.keras.Model(inputs=inputs, outputs=out)
    inference_model.compile(loss="sparse_categorical_crossentropy",
                            metrics="accuracy")
    return inference_model

In [ ]:
inference_model = get_inference_model(model)
inference_model.summary(expand_nested=True)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
tflite_model = converter.convert()
model_path = "model.tflite"

with open(model_path, 'wb') as f:
    f.write(tflite_model)
!zip submission.zip $model_path